In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nibabel as nib
import sklearn.model_selection
import sklearn.metrics
import pandas as pd

In [5]:
# Load brain imaging data using NiBabel
brain_data = nib.load("avg152T1_LR_nifti.hdr")
X = brain_data.get_fdata()
data_shape = X.shape

In [6]:
# Load labels depression diagnosis
data = pd.read_csv('updated_oasis_cross-sectional.csv')
y = data['depression_diagnosis'].replace({'not depressed': 0, 'likely depressed': 1}).values
np.save('oasis_cross-sectional.npy', y)
y = np.load('oasis_cross-sectional.npy', allow_pickle=True)
num_subjects = X.shape[0]
y = y[:num_subjects]

In [7]:
# Preprocess the data
X = X.reshape(num_subjects, *data_shape[1:])
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).long()
X = X / X.max()
X = X.reshape(num_subjects, -1)
X = X[:, :1000]
feature_dim = 1000

In [8]:
# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)


In [14]:
#DNN having 3layers with relu activation
class DNN(nn.Module):
    def __init__(self, feature_dim, hidden_dim, output_dim):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(feature_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.softmax(self.fc3(x))
        return x

In [15]:
#hyperparameters of the DNN
hidden_dim = 64 
output_dim = 2 
learning_rate = 0.01 
batch_size = 32 
epochs = 50 

In [16]:
#DNN object
model = DNN(feature_dim, hidden_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [17]:
#train the dnn
best_val_loss = float('inf') 
best_model = None 
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_acc = 0
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        optimizer.zero_grad()
        print(X_batch)
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (y_pred.argmax(dim=1) == y_batch).sum().item()
        if (i + 1) % 100 == 0:
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {loss.item():.4f}, Accuracy: {(y_pred.argmax(dim=1) == y_batch).sum().item() / batch_size:.4f}')
    train_loss /= len(X_train)
    train_acc /= len(X_train)
    model.eval()
    val_loss = 0
    val_acc = 0
    for i in range(0, len(X_val), batch_size):
        X_batch = X_val[i:i+batch_size]
        y_batch = y_val[i:i+batch_size]
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        val_loss += loss.item()
        val_acc += (y_pred.argmax(dim=1) == y_batch).sum().item()
    val_loss /= len(X_val)
    val_acc /= len(X_val)
    print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model


tensor([[0.0471, 0.0471, 0.0471,  ..., 0.0353, 0.0353, 0.0353],
        [0.0510, 0.0471, 0.0510,  ..., 0.0314, 0.0314, 0.0314],
        [0.0510, 0.0510, 0.0510,  ..., 0.0353, 0.0353, 0.0353],
        ...,
        [0.0471, 0.0510, 0.0471,  ..., 0.0314, 0.0314, 0.0314],
        [0.0510, 0.0510, 0.0510,  ..., 0.0353, 0.0314, 0.0314],
        [0.0627, 0.0667, 0.0706,  ..., 0.0314, 0.0314, 0.0275]])
tensor([[0.0549, 0.0588, 0.0588,  ..., 0.0314, 0.0314, 0.0314],
        [0.0549, 0.0588, 0.0588,  ..., 0.0314, 0.0314, 0.0314],
        [0.0549, 0.0627, 0.0588,  ..., 0.0314, 0.0275, 0.0314],
        ...,
        [0.0510, 0.0471, 0.0471,  ..., 0.0314, 0.0314, 0.0314],
        [0.0510, 0.0510, 0.0549,  ..., 0.0314, 0.0314, 0.0314],
        [0.0471, 0.0471, 0.0510,  ..., 0.0353, 0.0314, 0.0353]])
Epoch 1, Train Loss: 0.0258, Train Accuracy: 0.5741, Val Loss: 0.0379, Val Accuracy: 0.6111
tensor([[0.0471, 0.0471, 0.0471,  ..., 0.0353, 0.0353, 0.0353],
        [0.0510, 0.0471, 0.0510,  ..., 0.0314, 0

tensor([[0.0549, 0.0588, 0.0588,  ..., 0.0314, 0.0314, 0.0314],
        [0.0549, 0.0588, 0.0588,  ..., 0.0314, 0.0314, 0.0314],
        [0.0549, 0.0627, 0.0588,  ..., 0.0314, 0.0275, 0.0314],
        ...,
        [0.0510, 0.0471, 0.0471,  ..., 0.0314, 0.0314, 0.0314],
        [0.0510, 0.0510, 0.0549,  ..., 0.0314, 0.0314, 0.0314],
        [0.0471, 0.0471, 0.0510,  ..., 0.0353, 0.0314, 0.0353]])
Epoch 29, Train Loss: 0.0248, Train Accuracy: 0.6296, Val Loss: 0.0354, Val Accuracy: 0.6667
tensor([[0.0471, 0.0471, 0.0471,  ..., 0.0353, 0.0353, 0.0353],
        [0.0510, 0.0471, 0.0510,  ..., 0.0314, 0.0314, 0.0314],
        [0.0510, 0.0510, 0.0510,  ..., 0.0353, 0.0353, 0.0353],
        ...,
        [0.0471, 0.0510, 0.0471,  ..., 0.0314, 0.0314, 0.0314],
        [0.0510, 0.0510, 0.0510,  ..., 0.0353, 0.0314, 0.0314],
        [0.0627, 0.0667, 0.0706,  ..., 0.0314, 0.0314, 0.0275]])
tensor([[0.0549, 0.0588, 0.0588,  ..., 0.0314, 0.0314, 0.0314],
        [0.0549, 0.0588, 0.0588,  ..., 0.0314, 

In [18]:
#Evaluation of the dnn
best_model.eval()
test_loss = 0
test_acc = 0
for i in range(0, len(X_test), batch_size):
    X_batch = X_test[i:i+batch_size]
    y_batch = y_test[i:i+batch_size]
    y_pred = best_model(X_batch)
    loss = criterion(y_pred, y_batch)
    test_loss += loss.item()
    test_acc += (y_pred.argmax(dim=1) == y_batch).sum().item()
test_loss /= len(X_test)
test_acc /= len(X_test)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')


Test Loss: 0.0372, Test Accuracy: 0.5789
